In [1]:
import numpy as np
import pandas as pd
import re

In [2]:
df = pd.read_json('data/receitas.json')

In [3]:
df

,directions,fat,date,categories,calories,desc,protein,rating,title,ingredients,sodium
0,"[1. Place the stock, lentils, celery, carrot, ...",7.0,2006-09-01 04:00:00+00:00,"[Sandwich, Bean, Fruit, Tomato, turkey, Vegeta...",426.0,None,30.0,2.500,"Lentil, Apple, and Turkey Wrap","[4 cups low-sodium vegetable or chicken stock,...",559.0
1,[Combine first 9 ingredients in heavy medium s...,23.0,2004-08-20 04:00:00+00:00,"[Food Processor, Onion, Pork, Bake, Bastille D...",403.0,This uses the same ingredients found in boudin...,18.0,4.375,Boudin Blanc Terrine with Red Onion Confit,"[1 1/2 cups whipping cream, 2 medium onions, c...",1439.0
2,[In a large heavy saucepan cook diced fennel a...,7.0,2004-08-20 04:00:00+00:00,"[Soup/Stew, Dairy, Potato, Vegetable, Fennel, ...",165.0,None,6.0,3.750,Potato and Fennel Soup Hodge,"[1 fennel bulb (sometimes called anise), stalk...",165.0
3,[Heat oil in heavy large skillet over medium-h...,NaN,2009-03-27 04:00:00+00:00,"[Fish, Olive, Tomato, Sauté, Low Fat, Low Cal,...",NaN,The Sicilian-style tomato sauce has tons of Me...,NaN,5.000,Mahi-Mahi in Tomato Olive Sauce,"[2 tablespoons extra-virgin olive oil, 1 cup c...",NaN
4,[Preheat oven to 350°F. Lightly grease 8x8x2-i...,32.0,2004-08-20 04:00:00+00:00,"[Cheese, Dairy, Pasta, Vegetable, Side, Bake, ...",547.0,None,20.0,3.125,Spinach Noodle Casserole,"[1 12-ounce package frozen spinach soufflé, th...",452.0
...,...,...,...,...,...,...,...,...,...,...,...
20125,[Beat whites in a bowl with an electric mixer ...,2.0,2004-08-20 04:00:00+00:00,"[Mixer, Cheese, Egg, Fry, Cocktail Party, Parm...",28.0,None,2.0,3.125,Parmesan Puffs,"[2 large egg whites, 3 oz Parmigiano-Reggiano,...",64.0
20126,[Bring broth to simmer in saucepan.Remove from...,28.0,2008-02-28 22:06:54+00:00,"[Side, Kid-Friendly, High Fiber, Dinner, Parme...",671.0,Cooking the artichokes with the rice infuses t...,22.0,4.375,Artichoke and Parmesan Risotto,"[5 1/2 cups (or more) low-salt chicken broth, ...",583.0
20127,"[Using a sharp knife, cut a shallow X in botto...",38.0,2005-10-21 18:21:20+00:00,"[Onion, Poultry, turkey, Vegetable, Bake, Kid-...",563.0,None,31.0,4.375,Turkey Cream Puff Pie,"[1 small tomato, 1 small onion, finely chopped...",652.0
20128,[Heat 2 tablespoons oil in heavy medium skille...,24.0,2004-08-20 04:00:00+00:00,"[Milk/Cream, Citrus, Dairy, Fish, Garlic, Past...",631.0,"Sharon Hooykaas of Los Alamitos, California, w...",45.0,4.375,Snapper on Angel Hair with Citrus Cream,"[4 tablespoons olive oil, 4 shallots, thinly s...",517.0


In [4]:
df.describe()

,fat,calories,protein,rating,sodium
count,1.590800e+04,1.597600e+04,15929.000000,20100.000000,1.597400e+04
mean,3.460975e+02,6.307857e+03,99.946199,3.713060,6.211474e+03
std,2.043102e+04,3.585851e+05,3835.616663,1.343144,3.328903e+05
min,0.000000e+00,0.000000e+00,0.000000,0.000000,0.000000e+00
25%,7.000000e+00,1.980000e+02,3.000000,3.750000,8.000000e+01
50%,1.700000e+01,3.310000e+02,8.000000,4.375000,2.940000e+02
75%,3.300000e+01,5.860000e+02,27.000000,4.375000,7.110000e+02
max,1.722763e+06,3.011122e+07,236489.000000,5.000000,2.767511e+07


In [5]:
# Coletar as categorias das receitas mais calóricas
categories = df.sort_values('calories', ascending=False).categories.fillna("").to_list()
categories

[['Berry',
  'Fruit',
  'Nut',
  'Dessert',
  'Bake',
  'Christmas',
  'Thanksgiving',
  'Cranberry',
  'Pear',
  'Walnut',
  'Fall',
  'Winter',
  'Gourmet',
  'Vegetarian',
  'Pescatarian',
  'Peanut Free',
  'Soy Free',
  'Kosher'],
 ['Egg', 'Fruit', 'Bake', 'Blueberry', 'Summer', 'Tapioca', 'Gourmet'],
 ['Berry',
  'Fruit',
  'Nut',
  'Dessert',
  'Bake',
  'Thanksgiving',
  'Cranberry',
  'Apricot',
  'Walnut',
  'Fall',
  'Bon Appétit',
  'Vegetarian',
  'Pescatarian',
  'Wheat/Gluten-Free',
  'Peanut Free',
  'Soy Free',
  'Kosher'],
 ['Food Processor',
  'Backyard BBQ',
  'Ground Lamb',
  'Spice',
  'Summer',
  'Grill',
  'Grill/Barbecue',
  'Cinnamon',
  'Clove',
  'Parsley',
  'Gourmet'],
 ['Lamb',
  'Rice',
  'Vegetable',
  'Braise',
  'Dinner',
  'Raisin',
  'Meat',
  'Lamb Shank',
  'Root Vegetable',
  'Carrot',
  'Fall',
  'Winter',
  'Gourmet',
  'Wheat/Gluten-Free',
  'Peanut Free',
  'Tree Nut Free',
  'Soy Free',
  'No Sugar Added'],
 ['Lamb',
  'Rice',
  'Vegetable',

In [6]:
# Coletar todas as possíveis categorias
categories_list = []
for cat in categories:
    for c in cat:
        categories_list.append(c)
categories_list = pd.Series(categories_list).unique()
categories_list

array(['Berry', 'Fruit', 'Nut', 'Dessert', 'Bake', 'Christmas',
       'Thanksgiving', 'Cranberry', 'Pear', 'Walnut', 'Fall', 'Winter',
       'Gourmet', 'Vegetarian', 'Pescatarian', 'Peanut Free', 'Soy Free',
       'Kosher', 'Egg', 'Blueberry', 'Summer', 'Tapioca', 'Apricot',
       'Bon Appétit', 'Wheat/Gluten-Free', 'Food Processor',
       'Backyard BBQ', 'Ground Lamb', 'Spice', 'Grill', 'Grill/Barbecue',
       'Cinnamon', 'Clove', 'Parsley', 'Lamb', 'Rice', 'Vegetable',
       'Braise', 'Dinner', 'Raisin', 'Meat', 'Lamb Shank',
       'Root Vegetable', 'Carrot', 'Tree Nut Free', 'No Sugar Added',
       'Chocolate', 'Almond', 'Christmas Eve', 'Apple', 'Quick & Easy',
       'Pan-Fry', 'Couscous', 'Mushroom', 'Mustard', "Father's Day",
       'Lamb Chop', 'Sugar Conscious', 'Chicken', 'Beef', 'Beef Rib',
       'Red Wine', 'Duck', 'Onion', 'Poultry', 'Roast', 'Parsnip',
       'White Wine', 'Shallot', 'Dairy Free', 'Citrus', 'Ginger',
       'Super Bowl', 'Orange', 'Poker/Game Ni

In [7]:
# Criar um data frame com todas as possíveis categorias.
df_categories = pd.DataFrame({'categories': categories_list, 'count': 0}).set_index('categories')
df_categories

,count
categories,
Berry,0
Fruit,0
Nut,0
Dessert,0
Bake,0
...,...
Las Vegas,0
Camping,0
Egypt,0


In [8]:
# Contar categorias presentes nas 1000 receitas mais calóricas
for i in range(1000):
    for cat in categories[i]:
        df_categories.loc[cat, 'count'] += 1
df_categories

,count
categories,
Berry,8
Fruit,84
Nut,32
Dessert,143
Bake,235
...,...
Las Vegas,0
Camping,0
Egypt,0


In [9]:
#1. A categorias pertencem as comidas mais calóricas?
df_categories.sort_values('count', ascending=False).head()

,count
categories,
Bon Appétit,448
Peanut Free,387
Soy Free,368
Gourmet,363
Tree Nut Free,330


In [10]:
# Coletar os ingredientes das receitas mais calóricas
ingredientes = df.sort_values('calories', ascending=False).ingredients.fillna("").to_frame()
ingredientes

,ingredients
11444,"[2 firm-ripe Anjou pears, peeled, cored, and f..."
6400,"[1 1/4 cups packed light brown sugar, 5 tables..."
19749,"[1 cup (packed) golden brown sugar, 2 large eg..."
2985,"[3 slices firm white sandwich bread, torn into..."
1308,"[3 cups basmati rice (7 ounces), 1 medium onio..."
...,...
20090,"[1/4 pound spaghetti, 1/3 cup finely chopped s..."
20094,"[Poire Williams (clear pear brandy), 6 quarter..."
20104,"[3 tablespoons olive oil, divided, plus more, ..."
20113,"[Nonstick vegetable oil spray, 2 medium zucchi..."


In [11]:
unit_list = [
    r'[0-9]+/? ',
    r'^[0-9]+/?',
    r'cup[s]? ',
    r'teaspoon[s]? ',
    r'tablespoon[s]? ',
    r'large? ',
    r'fresh[ly]?[y]?',
    r's$',
    r'extra-virgin ',
    r'all[- ]purpose ',
    r', divided',
    r'chopped ',
    r'dried ',
    r'[ ]?ground [ ]?',
    r'coarse ',
    r'granulated ',
    r' white',
    r' yolk',
    r'firm-ripe ',
    r'peeled,',
    r'cored,',
    r'finely,',
    r'chopped',
    r'canned ',
    r'whole ',
    r'crushed', r'slices ', r'grain ', r'roll', r'into', r'cut',
]
def get_ingredients(ingredient):
    if ingredient:
        for unit in unit_list:
            ingredient = re.sub(unit, '', ingredient)
        #ingredient = re.split(r'(,)*( )*', ingredient)
    return ingredient

In [12]:
ingredientes.ingredients[0][0], get_ingredients(ingredientes.ingredients[0][0])

('4 cups low-sodium vegetable or chicken stock',
 'low-sodium vegetable or chicken stock')

In [13]:
# Coletar tods os possíveis ingredientes
ingredientes_list = []
for item in ingredientes.ingredients:   
    for subitem in item:
        subitem_list = get_ingredients(subitem.lower())
        #for s in subitem_list:
        ingredientes_list.append(subitem_list)
ingredientes_list = pd.Series(ingredientes_list).unique()
ingredientes_list

array(['anjou pears,   and finely ', 'sour cherrie', 'currant', ...,
       'eggplant,  crosswise  1/2-inch-thick slice',
       'soft  goat cheese, room temperature (about ounces)', 'bread or '],
      dtype=object)

In [14]:
# Criar um data frame com todas os possíveis ingredientes.
df_ingredientes = pd.DataFrame({'ingredients': ingredientes_list, 'count': 0}).set_index('ingredients')
df_ingredientes

,count
ingredients,
"anjou pears, and finely",0
sour cherrie,0
currant,0
cranberrie,0
packed light brown sugar,0
...,...
"spinach, coarsely",0
"medium zucchini, each lengthwise slice",0
"eggplant, crosswise 1/2-inch-thick slice",0


In [15]:
# Contar ingredientes presentes nas 1000 receitas mais calóricas
for i in range(1000):
    for item in ingredientes.ingredients[i]:
        df_ingredientes.loc[get_ingredients(item.lower()), 'count'] += 1
df_ingredientes

,count
ingredients,
"anjou pears, and finely",0
sour cherrie,0
currant,3
cranberrie,4
packed light brown sugar,11
...,...
"spinach, coarsely",0
"medium zucchini, each lengthwise slice",0
"eggplant, crosswise 1/2-inch-thick slice",0


In [16]:
#2. Quais os top 10 ingredientes contidos nas receitas mais calóricas?
df_ingredientes.sort_values('count', ascending=False).head(10)

,count
ingredients,
olive oil,307
sugar,260
salt,239
egg,171
black pepper,128
water,127
flour,121
lemon juice,116
kosher salt,96
